# Course Project - Convex Optimization for Signal Processing and Communications

- Student: Lucas von Ancken Garcia
- Matrikelnummer: 2576600 

In [1]:
import numpy as np
import os
import cvxpy as cvx

np.random.seed(1)
np.set_printoptions(precision=3, suppress=True, linewidth=np.inf)

## Auxillary Methods

In [2]:
# Generation of the complex Channel Matrix H
def generate_channel_matrix(m, n):
    real_part = np.random.normal(0, np.sqrt(0.5), (m,n))
    imag_part = np.random.normal(0, np.sqrt(0.5), (m,n))

    return real_part + 1j*imag_part

# Generate white noise vector
def generate_white_noise(n):
    real_part = np.random.normal(0, np.sqrt(0.5), n)
    imag_part = np.random.normal(0, np.sqrt(0.5), n)

    return real_part + 1j*imag_part

def generate_random_channel_vector(n, mean, std):
    real_part = np.random.normal(mean, std, n)
    imag_part = np.random.normal(mean, std, n)
    
    return (real_part + 1j*imag_part).reshape(n,1)

def matrix_square_root(R):
    # Eigenvalue decomposition. The eigenvectors are normalized
    eigenvalues, eigenvectors = np.linalg.eigh(R)
    
    # Calculate the square root of the diagonal eigenvalue matrix
    sqrt_eigenvalues = np.sqrt(eigenvalues)
    sqrt_eigenvalue_matrix = np.diag(sqrt_eigenvalues)
    
    # Reconstruct the square root matrix
    R_sqrt = eigenvectors @ sqrt_eigenvalue_matrix @ eigenvectors.T
    return R_sqrt

def generate_random_hermitian_psd_matrix(n):
    # Generate a random matrix A
    A = np.random.randn(n, n) + 1j*np.random.randn(n, n)
    
    # Construct a symmetric matrix by multiplying A with its transpose
    A = np.dot(A, A.conj().T)/5
    
    return A

def generate_random_symmetric_psd_matrix(n):
    # Generate a random matrix A
    A = np.random.randn(n, n)
    
    # Construct a symmetric matrix by multiplying A with its transpose
    A = np.dot(A, A.conj().T)
    
    return A

In [3]:
def save_results(filename, num_tx, num_rx, power_limit_per_tx, total_power, prob, objective, constraints, P, H, Rn, hc=None):
    # Create results directory if it does not exist
    if not os.path.exists('./results'):
        os.makedirs('./results')

    # Initialize the result string
    result = "SIMULATION OUTPUT FILE"

    # Append constants section
    result += "CONSTANTS\n\n"
    result += "Number of Transmitters: {}\n".format(num_tx)
    result += "Number of Receivers: {}\n".format(num_rx)
    result += "Power Limits per TX: {} (sum = {})\n".format(power_limit_per_tx, sum(power_limit_per_tx))
    result += "Total Power Limit: {}\n\n\n".format(total_power)

    # Append input parameters section
    result += "INPUT PARAMETERS\n\n"
    result += "Covariance Matrix Noise Rn = \n{}\n".format(Rn)
    result += "Channel Matrix  H = \n{}\n\n\n".format(H.value)

    # Append results section
    result += "\n\n--------------------\n"
    result += "RESULTS"
    result += "\n--------------------\n"
    result += "Status: {}\n".format(prob.status)
    result += "Num iterations: {}\n".format(prob.solver_stats.num_iters)
    result += "Solve Time: {}\n".format(prob.solver_stats.solve_time)
    result += "Solver name: {}\n".format(prob.solver_stats.solver_name)
    result += "Optimal value: {}\n".format(objective.value)
    result += "Num of Constraints: {}\n\n".format(len(constraints))
    result += "\n"
    result += "P* = \n{}\n".format(P.value)
    result += "Eigenvalues of P* = {}\n".format(np.linalg.eigvalsh(P.value))
    result += "Actual Total Power of the Optimal Solution: {}\n".format(P.value.trace())
    if hc is not None:
        interference = np.trace(P.value @ (hc @ hc.conjugate().transpose()))
        result += "Interference in new user: {}\n".format(interference)
    result += "\n\n--------------------\n"
    result += "Lagrange Multipliers"
    result += "\n--------------------\n"
    result += "For P >= 0: \n{}\n".format(constraints[0].dual_value)
    result += "Eigenvalues of Lagrange Multiplier of P >=0: {}\n".format(np.linalg.eigvalsh(constraints[0].dual_value))
    result += "For total power:\n{}\n".format(constraints[1].dual_value)
    result += "For the limit power of each TX:\n{}\n".format([x.dual_value for x in constraints[2:]])
    if hc is not None:
        result += "For the zero interference in new user: \n{}\n".format(constraints[-1].dual_value)

    # Print the result to the console
    print(result)

    # Save the result to a file
    with open(os.path.join('./results', filename), 'w') as file:
        file.write(result)
    
def save_average_results(filename, Rn_avg, H_avg, optimal_value_avg, P_avg, num_iter):
    # Create the results string
    results = []
    
    results.append("AVERAGE INPUT PARAMETERS\n")
    results.append("\n")
    results.append("Average Noise Covariance Matrix Rn = {}\n".format(Rn_avg))
    results.append("Average Channel Matrix H = \n")
    results.append("{}\n".format(H_avg))
    results.append("\n")
    results.append("\n")

    results.append("RESULTS\n")
    results.append("\n")
    results.append("Number of averaged iterations: {}\n".format(num_iter))
    results.append("Optimal Value Average: {}\n".format(optimal_value_avg))
    results.append("Average P* = \n")
    results.append("{}\n".format(P_avg))

    # Join the results list into a single string
    results_string = "".join(results)

    # Print the results string to the screen
    print(results_string)

    # Ensure the directory exists
    os.makedirs("./results", exist_ok=True)

    # Save the results string to the specified file
    file_path = os.path.join("./results", filename)
    with open(file_path, 'w') as file:
        file.write(results_string)

## Problem 1

Given a single-user (point-to-point) MIMO communication channel with the channel matrix $H$ and zero mean
colored noise $n$ at the receiver antennas, the received symbol at the receiver is $y = H^H \cdot x + n$, where $x$ is the
transmitted symbol with zero mean. We want to **design the optimal transmit covariance matrix** that **maximizes the transmission rate** of the MIMO system. We assume that some of the **antennas have power constraints** that
restrict the transmitted power. Furthermore, **a sum power constraint applies**.

Generate required data to implement the problem formulation in CVX. Average the simulation results with Monte Carlo iterations.

In [4]:
VerboseMode = False

num_tx = 6  # Number of transmit antennas
num_rx = 4  # Number of receive antennas

total_power = 65

power_limit_per_tx = [9, 18, 10, 8, 12, 16]


In [5]:

H = cvx.Parameter((num_tx, num_rx), complex=True)
Rn_sqr_inv = cvx.Parameter((num_rx, num_rx), hermitian=True)

P = cvx.Variable((num_tx, num_tx), hermitian=True)

objective = cvx.Maximize(cvx.log_det(np.identity(num_rx) + Rn_sqr_inv @ cvx.conj(H.T) @ P @ H @ Rn_sqr_inv))

constraints = [P >> 0, cvx.real(cvx.trace(P)) <= total_power]

for i in range(num_tx):
    constraints.append(cvx.real(P[i, i]) <= power_limit_per_tx[i])

prob = cvx.Problem(objective, constraints)

In [6]:

# Rn_sqr = generate_random_symmetrical_psd_matrix(num_rx)
Rn_sqr = generate_random_hermitian_psd_matrix(num_rx)
Rn = Rn_sqr @ Rn_sqr

H.value = generate_channel_matrix(num_tx, num_rx)
Rn_sqr_inv.value = np.linalg.inv(Rn_sqr)

H_used_first_iteration = np.copy(H.value)
Rn_sqr_inv_used_first_iteration = np.copy(Rn_sqr_inv.value)

# For optimal results, not inaccurate, uncomment command below
prob.solve(verbose=VerboseMode, max_iters=int(10e6))
# prob.solve(verbose=VerboseMode)

c:\Users\User\anaconda3\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:237: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html
  warnings.warn(DPP_ERROR_MSG)


23.214265498443012

### Problem 1 results for first iteration

In [7]:
save_results("problem_1_first_iteration_10e5.txt", num_tx, num_rx, power_limit_per_tx, total_power, prob, objective, constraints, P, H, Rn)

SIMULATION OUTPUT FILECONSTANTS

Number of Transmitters: 6
Number of Receivers: 4
Power Limits per TX: [9, 18, 10, 8, 12, 16] (sum = 73)
Total Power Limit: 65


INPUT PARAMETERS

Covariance Matrix Noise Rn = 
[[3.233+0.j    2.31 +3.628j 1.95 -2.023j 0.12 +0.368j]
 [2.31 -3.628j 9.704+0.j    2.357-3.332j 2.855+1.571j]
 [1.95 +2.023j 2.357+3.332j 5.133+0.j    1.897+1.29j ]
 [0.12 -0.368j 2.855-1.571j 1.897-1.29j  1.99 +0.j   ]]
Channel Matrix  H = 
[[-0.486+0.593j -0.598+0.658j -0.475+0.202j -0.009+0.626j]
 [-0.79 -0.533j  0.166+0.886j  1.174+0.363j  0.525-0.211j]
 [-0.136+0.345j -0.628-0.053j -0.528+0.8j    1.197+1.075j]
 [ 0.036+1.545j -0.45 -0.987j  0.135-1.021j  1.485-0.357j]
 [ 0.085+0.113j  0.436+0.62j   0.212+0.223j -0.249-1.43j ]
 [-0.808-0.217j -0.247+0.585j -0.148+0.163j  0.415+0.539j]]




--------------------
RESULTS
--------------------
Status: optimal
Num iterations: 3080650
Solve Time: 849.7130302
Solver name: SCS
Optimal value: 23.214265498443012
Num of Constraints: 8


P

### Problem 1 results for averaged iterations

In [8]:
P_list = [P.value]
H_list = [H.value]
Rn_list = [Rn]
optimal_value_list = [prob.value]


for i in range(9):
    H.value = generate_channel_matrix(num_tx, num_rx)
    
    Rn_sqr = generate_random_hermitian_psd_matrix(num_rx)
    # Rn_sqr = generate_random_symmetric_psd_matrix(num_rx)
    Rn = Rn_sqr @ Rn_sqr

    power_noise = np.random.uniform(low=0, high=5)
    Rn_sqr_inv.value = np.linalg.inv(Rn_sqr)

    prob.solve()

    H_list.append(H.value)
    Rn_list.append(Rn)

    P_list.append(P.value)
    optimal_value_list.append(prob.value)

H_avg = sum(H_list)/len(H_list)
Rn_avg = sum(Rn_list)/len(Rn_list)

P_avg = sum(P_list)/len(P_list)
optimal_value_avg = sum(optimal_value_list)/len(optimal_value_list)

c:\Users\User\anaconda3\Lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [9]:
save_average_results("problem_1_avg_results.txt", Rn_avg, H_avg, optimal_value_avg, P_avg, len(H_list))

AVERAGE INPUT PARAMETERS

Average Noise Covariance Matrix Rn = [[ 3.49 +0.j     0.307+0.051j -0.437-0.371j -0.584-0.262j]
 [ 0.307-0.051j  4.282+0.j     0.239-0.119j  0.721+0.868j]
 [-0.437+0.371j  0.239+0.119j  3.049+0.j     0.737+0.902j]
 [-0.584+0.262j  0.721-0.868j  0.737-0.902j  4.685+0.j   ]]
Average Channel Matrix H = 
[[-0.447-0.005j  0.13 -0.279j -0.034+0.477j  0.199+0.045j]
 [ 0.233+0.291j  0.039-0.128j  0.021+0.016j  0.572-0.429j]
 [ 0.118+0.09j  -0.16 +0.129j  0.26 +0.364j -0.162+0.35j ]
 [-0.125+0.259j  0.083-0.265j -0.406-0.145j  0.436+0.408j]
 [-0.243+0.115j  0.096+0.3j   -0.082-0.206j  0.518-0.296j]
 [-0.321-0.419j -0.453+0.186j  0.07 +0.176j -0.011+0.008j]]


RESULTS

Number of averaged iterations: 10
Optimal Value Average: 19.498752973118272
Average P* = 
[[ 8.217+0.j    -0.258-0.285j -0.986+1.11j   0.628-0.211j  0.586-1.262j  0.114-1.422j]
 [-0.258+0.285j 13.27 +0.j    -0.299-1.108j -0.41 -0.579j -0.002+0.104j -1.105-0.007j]
 [-0.986-1.11j  -0.299+1.108j  9.68 +0.j  

## Problem 2
We assume further that a single-antenna co-channel user with channel vector hc is now present. The received symbol is $y_c = h^H_c \cdot x + n_c$, where $n_c$ is the zero mean colored noise at this user. We want that this user does not receive any interference in expectation, i.e., $E[∥y_c∥^2] = 0$.

Please modify the problem formulation to adopt the new constraint and solve it in CVX.

In [10]:
hc = generate_random_channel_vector(num_tx, 0, 4)
# hc = np.real(generate_random_channel_vector(num_tx, 0, 4))

In [11]:
new_constraints = prob.constraints + [cvx.trace(cvx.real((hc @ hc.conjugate().transpose()) @ P)) == 0]

H.value = H_used_first_iteration
Rn_sqr_inv.value = Rn_sqr_inv_used_first_iteration

new_prob = cvx.Problem(objective, new_constraints)
new_prob.solve(verbose=VerboseMode)

c:\Users\User\anaconda3\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:237: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html
  warnings.warn(DPP_ERROR_MSG)
c:\Users\User\anaconda3\Lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


22.911843572915323

### Problem 2 Results

In [12]:
Rn_sqrt = np.linalg.inv(Rn_sqr_inv.value)
Rn = Rn_sqr @ Rn_sqr

# print_simulation_results(num_tx, num_rx, power_limit_per_tx, total_power, new_prob, objective, new_constraints, P, H, Rn, hc)
save_results("problem2_SCS_10e5.txt", num_tx, num_rx, power_limit_per_tx, total_power, new_prob, objective, new_constraints, P, H, Rn, hc)

SIMULATION OUTPUT FILECONSTANTS

Number of Transmitters: 6
Number of Receivers: 4
Power Limits per TX: [9, 18, 10, 8, 12, 16] (sum = 73)
Total Power Limit: 65


INPUT PARAMETERS

Covariance Matrix Noise Rn = 
[[ 4.218+0.j    -2.037-2.944j -1.345-1.289j  2.443-0.539j]
 [-2.037+2.944j  4.181+0.j     1.179-0.749j -0.755+1.436j]
 [-1.345+1.289j  1.179+0.749j  1.85 +0.j     0.967+1.391j]
 [ 2.443+0.539j -0.755-1.436j  0.967-1.391j  4.469+0.j   ]]
Channel Matrix  H = 
[[-0.486+0.593j -0.598+0.658j -0.475+0.202j -0.009+0.626j]
 [-0.79 -0.533j  0.166+0.886j  1.174+0.363j  0.525-0.211j]
 [-0.136+0.345j -0.628-0.053j -0.528+0.8j    1.197+1.075j]
 [ 0.036+1.545j -0.45 -0.987j  0.135-1.021j  1.485-0.357j]
 [ 0.085+0.113j  0.436+0.62j   0.212+0.223j -0.249-1.43j ]
 [-0.808-0.217j -0.247+0.585j -0.148+0.163j  0.415+0.539j]]




--------------------
RESULTS
--------------------
Status: optimal_inaccurate
Num iterations: 100000
Solve Time: 27.8972766
Solver name: SCS
Optimal value: 22.911843572915323


In [13]:
new_prob.solve(max_iters=int(1e6), verbose=VerboseMode)

In [ ]:
save_results("problem2_SCS_10e6.txt", num_tx, num_rx, power_limit_per_tx, total_power, new_prob, objective, new_constraints, P, H, Rn, hc)

SIMULATION OUTPUT FILECONSTANTS

Number of Transmitters: 6
Number of Receivers: 4
Power Limits per TX: [9, 18, 10, 8, 12, 16] (sum = 73)
Total Power Limit: 65


INPUT PARAMETERS

Covariance Matrix Noise Rn = 
[[ 4.218+0.j    -2.037-2.944j -1.345-1.289j  2.443-0.539j]
 [-2.037+2.944j  4.181+0.j     1.179-0.749j -0.755+1.436j]
 [-1.345+1.289j  1.179+0.749j  1.85 +0.j     0.967+1.391j]
 [ 2.443+0.539j -0.755-1.436j  0.967-1.391j  4.469+0.j   ]]
Channel Matrix  H = 
[[-0.486+0.593j -0.598+0.658j -0.475+0.202j -0.009+0.626j]
 [-0.79 -0.533j  0.166+0.886j  1.174+0.363j  0.525-0.211j]
 [-0.136+0.345j -0.628-0.053j -0.528+0.8j    1.197+1.075j]
 [ 0.036+1.545j -0.45 -0.987j  0.135-1.021j  1.485-0.357j]
 [ 0.085+0.113j  0.436+0.62j   0.212+0.223j -0.249-1.43j ]
 [-0.808-0.217j -0.247+0.585j -0.148+0.163j  0.415+0.539j]]




--------------------
RESULTS
--------------------
Status: optimal_inaccurate
Num iterations: 1000000
Solve Time: 250.4317311
Solver name: SCS
Optimal value: 23.06260734738794

In [ ]:
new_prob.solve(solver=cvx.CLARABEL, verbose=VerboseMode)

save_results("problem2_CLARABEL.txt", num_tx, num_rx, power_limit_per_tx, total_power, new_prob, objective, new_constraints, P, H, Rn, hc)